In [1]:
!pip install git+https://github.com/finetuneanon/transformers@gpt-neo-dungeon-localattention1
!nvidia-smi

  Cloning https://github.com/finetuneanon/transformers (to revision gpt-neo-dungeon-localattention1) to /tmp/pip-req-build-gc44z4o7
  Running command git clone -q https://github.com/finetuneanon/transformers /tmp/pip-req-build-gc44z4o7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2112854 sha256=b75a1c43ce3f2a2fa87f0c2f530702116400b753ad73bb5e843fa20234ce93f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-2pxd7jn_/wheels/0b/78/87/fcc1dd081713d41c282b4a209e484f9a7dfabb0af14127257e
Successfully built transformers
Tue Apr 27 16:56:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-

In [1]:
from transformers import GPTNeoForCausalLM, AutoConfig
import torch

config = AutoConfig.from_pretrained("EleutherAI/gpt-neo-2.7B")
# extra_layers = 26                 # 58 layers
# extra_heads = 4                   # 24 heads, 3072 hidden
# Parameter count: 6.730748928
# extra_layers = 8                  # 40 layers
# extra_heads = 8                   # 28 heads, 3584 hidden
# Parameter count: 6.354528768
extra_layers = 0                  # 32 layers
extra_heads = 12                  # 32 heads, 4096 hidden, GPT-3 6.7B
# Parameter count: 6.658011136
config.attention_layers.extend(["global", "local"] * (extra_layers // 2))
config.attention_types[0][0][1] = 16 + (extra_layers // 2)
config.num_layers = 32 + extra_layers
config.num_heads = 20 + extra_heads
config.hidden_size = 128 * config.num_heads

In [3]:
model = GPTNeoForCausalLM(config).half().cuda()
print("Parameter count: " + str(sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000000))
model = model.eval()

Parameter count: 6.658011136


In [3]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated() / 1024. / 1024.)
with torch.no_grad():
  for i in range(10):
    ids = torch.zeros(1,2048).int().cuda()
    output = model(ids)
    del ids
    del output.loss
    del output.logits
    del output
print(torch.cuda.max_memory_allocated() / 1024. / 1024.)
!nvidia-smi | grep MiB

12763.1640625
14687.18798828125
| N/A   61C    P0   175W / 250W |  15973MiB / 16280MiB |    100%      Default |


In [ ]:
# full checkpoints won't fit into colab system memory, but per-layer checkpoints should work
# can be easily converted on a system with more RAM before uploading to google drive
import os

def save(model):
    try: os.mkdir("chpt")
    except: pass
    checkpoint = {}
    for i, x in enumerate(model.state_dict().items()):
        checkpoint[x[0]] = f"chpt/b{i}.pt"
        torch.save(x[1], f"chpt/b{i}.pt")
    torch.save(checkpoint, f"chpt/m.pt")

In [6]:
# save dummy weights
print("save", flush=True)
save(model)
print("ok")

save
ok


In [2]:
# restart runtime and only run the config cell, then this, then the inference cell
import collections

class Checkpoint(collections.MutableMapping):
    def __init__(self):
        self.checkpoint = torch.load("chpt/m.pt")
    def __len__(self):
        return len(self.checkpoint)
    def __getitem__(self, key):
        return torch.load(self.checkpoint[key])
    def __setitem__(self, key, value):
        return
    def __delitem__(self, key, value):
        return
    def keys(self):
        return self.checkpoint.keys()
    def __iter__(self):
        for key in self.checkpoint:
            yield (key, self.__getitem__(key))
    def __copy__(self):
        return LazyDict()
    def copy(self):
        return LazyDict()

print("load", flush=True)
model = GPTNeoForCausalLM.from_pretrained(pretrained_model_name_or_path=None, config=config, state_dict=Checkpoint())
print("ok")

load


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  after removing the cwd from sys.path.


ok
